### Use NSGA-II genetic algorithm and trained model to find the best HEA composition (High CRSS and High YM)

#### loaddata and transfer to torch loader

In [1]:
import torch
import torch.utils.data as td
from sklearn.model_selection import train_test_split
import pandas as pd

def loaddata(pathway, support_list, imports_num, batchsize):
    df = pd.read_csv(pathway, header=None)
    x = df.iloc[:, 0:imports_num].values
    y = df.iloc[:, imports_num].values
    
    x_support = df.iloc[support_list, 0:imports_num].values
    y_support = df.iloc[support_list, imports_num].values
   
    # split data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=10)
   
    # create loader for the training data and labels
    train_x = torch.Tensor(x_train).float()
    train_x = train_x.reshape(x_train.shape[0], 1, imports_num).float()
    train_y = torch.Tensor(y_train).float()
    

    train_data = td.TensorDataset(train_x, train_y)
    train_loader = td.DataLoader(train_data, batch_size=batchsize, shuffle=False, num_workers=1)

    numtest = x.shape[0] - x_train.shape[0]
    #print(numtest)
    # create loader for the testing data and labels
    test_x = torch.Tensor(x_test).float()
    test_x = test_x.reshape(numtest, 1, imports_num).float()
    test_y = torch.Tensor(y_test).float()

    test_data = td.TensorDataset(test_x, test_y)
    test_loader = td.DataLoader(test_data, batch_size=batchsize, shuffle=False, num_workers=1)

    # create loader for the support data and labels
    support_x = torch.Tensor(x_support).float()
    support_x = support_x.reshape(len(support_list), 1, imports_num).float()
    support_y = torch.Tensor(y_support).float()

    support_data = td.TensorDataset(support_x, support_y)
    support_loader = td.DataLoader(support_data, batch_size=1, shuffle=False, num_workers=1)

    return train_loader, test_loader, support_loader


#### tranfor the fraction to description data

In [2]:
import numpy as np

class databuild():
    def __init__(self, fraction):
        self.frac_list = fraction
        self.rad_list = [192, 163, 189, 194, 197] # radius, unit: [pm]
        self.chi_list = [1.88, 1.91, 1.83, 1.66, 1.55] # Electronegativity
        self.VEC_list = [9, 10, 6, 8, 7] # valence electron concentration (containing d orbital electrons)
        self.E_list = [209, 200, 279, 211, 198] # Young's modulus [GPa]
        self.T_list = [1768, 1455, 1907, 1538, 1246] # unit: [°C]
        self.H_list = [0, -4, -1, -5, -7, -2, -8, -1, 2, 0] # mix entropy
        self.mu_list = [0.320, 0.312, 0.210, 0.291, 0.24] # Poisson ratio
        self.Ec_list = [4.39, 4.44, 4.10, 4.13, 2.92] # cohesive energy
        self.G_list = [75, 76, 115, 82, 76.4] # shear modulus [GPa]

    def fraction(self):
        fraction = []
        for num in self.frac_list:
            numb = num / sum(self.frac_list)
            fraction.append(numb)
        return fraction
    
    def delta_r(self):
        r_mean = np.mean(self.rad_list)
        delta_r = 0
        for rad, frac in zip(self.rad_list, self.fraction()):
            delta_r += frac * (1 - rad/r_mean) ** 2
        return np.sqrt(delta_r) * 10

    def delta_chi(self):
        chi_mean = np.mean(self.chi_list) # the mean value of electronegativity
        delta_chi = 0
        for chi_ele, frac in zip(self.chi_list, self.fraction()):
            delta_chi += frac * (chi_ele - chi_mean) ** 2
        return np.sqrt(delta_chi) * 10

    def VEC(self):
        VEC = 0
        for VEC_e, frac in zip(self.VEC_list, self.fraction()):
            VEC += frac * VEC_e
        return VEC

    def delta_H(self):
        index, H = 0, 0
        for loopi in range(len(self.fraction())):
            for loopj in range(loopi + 1, len(self.fraction())):
                H += 4 * self.H_list[index] * self.fraction()[loopi] * self.fraction()[loopj]
                index += 1
        return H

    def delta_S(self):
        R, S = 8.3145, 0
        for frac in self.fraction():
            if frac == 0:
                S += 0
            else:
                S += frac * np.log(frac)
        return -1* R * S

    def omega(self):
        return np.mean(self.T_list) * (self.delta_S() / (np.abs(self.delta_H()) + 1e-8)) / 500

    def Lambda(self):
        return self.delta_S() / (self.delta_r() ** 2 + 1e-8)

    def DX(self):
        DX = 0
        for loopi in range(len(self.fraction())):
            for loopj in range(loopi, len(self.fraction())):
                DX += self.fraction()[loopi] * self.fraction()[loopj] * np.abs(self.chi_list[loopi] - self.chi_list[loopj])
        return DX * 50

    def Ec(self):
        EC = 0
        for frac, Ec_ele in zip(self.fraction(), self.Ec_list):
            EC += frac * Ec_ele
        return EC
    
    def eta(self):
        G = self.G()
        eta = 0
        for frac, G_ele in zip(self.fraction(), self.G_list):
            eta += (frac * 2 * (G_ele - G) / (G_ele + G)) / \
            (1 + 0.5 * np.abs((frac * 2 * (G_ele - G) / (G_ele + G))))
        return eta

    def Dr(self):
        Dr = 0
        for loopi in range(len(self.fraction())):
            for loopj in range(loopi, len(self.fraction())):
                Dr += self.fraction()[loopi] * self.fraction()[loopj] * np.abs(self.rad_list[loopi] - self.rad_list[loopj])
        return Dr

    def A(self):
        mu = np.mean(self.mu_list)
        A = self.G() * self.delta_r() * (1 + mu) * (1 - mu)
        return A

    def F(self):
        return (2 * self.G()) / (1 - np.mean(self.mu_list)) / 10

    def G(self): 
        G = 0
        for frac, G_ele in zip(self.fraction(), self.G_list):
            G += frac * G_ele
        return G / 10

    def delta_G(self):
        G = self.G()
        delta_G = 0
        for frac, G_ele in zip(self.fraction(), self.G_list):
            delta_G += frac * (1 - G_ele / G) ** 2
        return np.sqrt(delta_G)

    def DG(self):
        DG = 0
        for loopi in range(len(self.fraction())):
            for loopj in range(loopi, len(self.fraction())):
                DG += self.fraction()[loopi] * self.fraction()[loopj] * np.abs(self.G_list[loopi] -self.G_list[loopj])
        return DG

    def mu(self):
        mu = np.mean(self.E_list) * self.delta_r()
        return 0.5 * mu / 10

    def descripters(self):
        des1 = np.array([self.delta_r(), self.delta_chi(), self.VEC(), self.delta_H(), 
                         self.delta_S(), self.omega(), self.Lambda(), self.DX(), 
                         self.Ec(), self.eta(), self.Dr(), self.A(), self.F(), self.G(), 
                         self.delta_G(), self.DG(), self.mu()])
        frac1 = self.fraction()
        return np.hstack([des1, frac1])
        
x = [0, 0, 0, 4, 5]
dat = databuild(x).descripters()
H0 = databuild(x).delta_H() - 7
H0


-7.0

#### trained model

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class CNNmodel(nn.Module):
    def __init__(self, inputs_num):
        super(CNNmodel, self).__init__()
        self.inputs_num = inputs_num
        self.conv0 = nn.Conv1d(2, 32, 1)
        self.conv1 = nn.Conv1d(32, 32, 1)
        self.conv2 = nn.Conv1d(32, 32, 1)
        self.conv3 = nn.Conv1d(32, 1, 1)

    def forward(self, x):
        x = F.relu(self.conv0(x))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten
        x = x.view(-1, self.inputs_num)
        return x

class DNNmodel(nn.Module):
    def __init__(self, outputs_num):
        super(DNNmodel, self).__init__()
        self.outputs_num = outputs_num
        self.fc0 = nn.Linear(outputs_num, 32)
        self.fc1 = nn.Linear(32, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class FSLmodel(nn.Module):
    def __init__(self, inputs_num, x1, x2, x3, x4, x5):
        super(FSLmodel, self).__init__()
        self.x1, self.x2, self.x3, self.x4, self.x5 = x1, x2, x3, x4, x5
        self.outputs_num = inputs_num * 5
        self.Cnet = CNNmodel(inputs_num)
        self.Dnet = DNNmodel(self.outputs_num)
        
    def forward(self, x):
        batchsize = x.shape[0]
        out1 = self.Cnet(torch.cat([x, self.x1.repeat(batchsize, 1, 1)], dim=1))
        out2 = self.Cnet(torch.cat([x, self.x2.repeat(batchsize, 1, 1)], dim=1))
        out3 = self.Cnet(torch.cat([x, self.x3.repeat(batchsize, 1, 1)], dim=1))
        out4 = self.Cnet(torch.cat([x, self.x4.repeat(batchsize, 1, 1)], dim=1))
        out5 = self.Cnet(torch.cat([x, self.x5.repeat(batchsize, 1, 1)], dim=1))

        out = torch.cat([out1, out2, out3, out4, out5], dim=1)
        outputs = self.Dnet(out)
        return outputs


#### function to predict the value of YM and CRSS

In [4]:
import torch

class function():
    def __init__(self):
        #self.index_to_removeYM = [14, 15]
        #self.imports_YM = 22-len(self.index_to_removeYM)
        self.imports_YM = 20
        pathway_YM = './Data/selected_YMdata.csv'
        support_listYM = [0, 8, 17, 30, 49]

        #self.index_to_removeCRSS = [10, 12, 13, 14, 15, 19]
        self.imports_CRSS = 16
        pathway_CRSS = './Data/selected_CRSSdata.csv'
        support_listCRSS = [0, 9, 18, 22, 31]
        
        _, _, support_YM = loaddata(pathway_YM, support_listYM, self.imports_YM, 1)
        _, _, support_CRSS = loaddata(pathway_CRSS, support_listCRSS, self.imports_CRSS, 1)

        sx_YM, sx_CRSS = [], []
        for supportx, supporty in support_YM:
            sx_YM.append(supportx)

        for supportx, supporty in support_CRSS:
            sx_CRSS.append(supportx)
        
        self.model_YM = FSLmodel(self.imports_YM, sx_YM[0], sx_YM[1], sx_YM[2], sx_YM[3], sx_YM[4])
        self.model_YM.load_state_dict(torch.load("./Models/HEA_YM.pt"))

        self.model_CRSS = FSLmodel(self.imports_CRSS, sx_CRSS[0], sx_CRSS[1], sx_CRSS[2], sx_CRSS[3], sx_CRSS[4])
        self.model_CRSS.load_state_dict(torch.load("./Models/HEA_CRSS.pt"))

    # function for predict YM
    @torch.no_grad()
    def func_YM(self, x):
        samples_num = x.shape[0]
        samples = np.zeros([samples_num, self.imports_YM])
        
        for i in range(samples_num):
            datai = databuild(x[i,:]).descripters()

        self.model_YM.eval()
        
        outputs = []
        for datax in samples:
           
            datax = torch.Tensor(datax).float()
            datax = datax.reshape(1, 1, self.imports_YM).float()
           
            out = self.model_YM(datax)[0]
            outputs.append(out)

        return outputs

    @torch.no_grad()
    def func_CRSS(self, x):
        samples_num = x.shape[0]
        samples = np.zeros([samples_num, self.imports_CRSS])
        
        for i in range(samples_num):
            datai = databuild(x[i,:]).descripters()

        self.model_CRSS.eval()
        
        outputs = []
        for datax in samples:
           
            datax = torch.Tensor(datax).float()
            datax = datax.reshape(1, 1, self.imports_CRSS).float()
           
            out = self.model_CRSS(datax)[0]
            outputs.append(out)

        return outputs


x = np.random.rand(4,5)
outputs1 = function().func_YM(x)
outputs2 = function().func_CRSS(x)
print(outputs1)
print(outputs2)


[tensor([211.4205]), tensor([211.4205]), tensor([211.4205]), tensor([211.4205])]
[tensor([2.5120]), tensor([2.5120]), tensor([2.5120]), tensor([2.5120])]


#### parameter used in problem function, and randomly form the samples

In [5]:
class defparameter():
    def __init__(self, fraction):
        self.fraction = fraction
        self.fraction_num = self.fraction.shape[0]
        self.rad_list = [192, 163, 189, 194, 197] # radius, unit: [pm]
        self.VEC_list = [9, 10, 6, 8, 7] # valence electron concentration (containing d orbital electrons)
        self.H_list = [0, -4, -1, -5, -7, -2, -8, -1, 2, 0] # mix entropy

    def H(self):
        H = np.zeros(self.fraction_num)
        for idx in range(self.fraction_num):
            index = 0
            for loopi in range(5):
                for loopj in range(loopi+1, 5):
                    H[idx] += 4 * self.H_list[index] * self.fraction[idx, loopi] * self.fraction[idx, loopj]
                    index += 1

        return H

    def VEC(self):
        VEC = np.zeros(self.fraction_num)
        for idx in range(self.fraction_num):
            for VEC_ele, frac in zip(self.VEC_list, self.fraction[idx, :]):
                VEC[idx] += frac * VEC_ele
        return VEC

    def delta_r(self):
        delta_r = np.zeros(self.fraction_num)
        for idx in range(self.fraction_num):
            r = 0 
            for rad, frac in zip(self.rad_list, self.fraction[idx,:]):
                r += rad * frac
            
            temp = 0
            for rad, frac in zip(self.rad_list, self.fraction[idx,:]):
                temp += frac * (1 - rad / r) ** 2
            
            delta_r[idx] = np.sqrt(temp)
            
        return delta_r * 100
        

def gen_samples(sample_num):
    fraction_norm = np.zeros([sample_num, 5])
    index = 0
    while index != sample_num:
        fraction_norm_temp = np.random.random([1, 5])
        fraction_norm_temp /= np.sum(fraction_norm_temp)
        
        if defparameter(fraction_norm_temp).H() <= 7 and \
        defparameter(fraction_norm_temp).H() >= -22 and \
        defparameter(fraction_norm_temp).VEC() >= 8 and \
        defparameter(fraction_norm_temp).delta_r() <= 6.6:
            
            fraction_norm[index,:] += fraction_norm_temp.reshape(5,)
            index += 1
            
    return fraction_norm

frac = gen_samples(10)


#### define the problem used in optimization

In [8]:
# define the problem
from pymoo.core.problem import Problem
import autograd.numpy as anp

class problem_to_solve(Problem):
    def __init__(self):
        super().__init__(
            n_var = 5,  # number of variable (5 element here)
            n_obj = 2,  # two objectives (YM and CRSS here)
            n_constr = 4,
            x1 = anp.array([0, 0, 0, 0, 0]),
            xu = anp.array([100, 100, 100, 100, 100])
        )

        # load the trained model
        self.function = function()

    # minimize problem
    def _evaluate(self, x, out, *args, **kwargs):
        
        # objective function 0
        f0 = -1 * self.function.func_YM(x)
        # objective function 1
        f1 = -1 * self.function.func_CRSS(x)

        H0 = defparameter(x).H() - 7
        H1 = -1 * defparameter(x).H() -22
        VEC = -1 * defparameter(x).VEC() + 8
        delta_r = defparameter(x).delta_r() - 6.6

        out["F"] = anp.array([f0, f1])
        out["G"] = anp.array([H0, H1, VEC, delta_r])

problem_to_solve()

#### optimize the problem

In [9]:
from pymoo.algorithms.moo.nsga2 import NSGA2

from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation

from pymoo.optimize import minimize


def opt_algorithm():
    
    problem = problem_to_solve()
    
    algorithm = NSGA2(
        pop_size=100,
        sampling=BinaryRandomSampling(),
        crossover=TwoPointCrossover(),
        mutation=BitflipMutation(),
        eliminate_duplicates=True
    )

    res = minimize(
        problem,
        algorithm,
        ('n_gen', 100),
        seed = 256,
        verbose=False
    )

    # Access result
    YM_x, CRSS_x = res.X, res.X
    print(YM_x)
   

    #use the trained model to predict the value of YM and CRSS
    #YM_pred_opt = function().func_YM(YM_X)
    #CRSS_pred_opt = container.func_CRSS(CRSS_x)
    
    #return YM_pred_opt, CRSS_pred_opt

opt_algorithm()

C:\Users\lpwu\AppData\Local\Temp\ipykernel_100360\2203665942.py:19: RuntimeWarning: invalid value encountered in divide
  numb = num / sum(self.frac_list)
C:\Users\lpwu\AppData\Local\Temp\ipykernel_100360\1847673077.py:36: RuntimeWarning: divide by zero encountered in scalar divide
  temp += frac * (1 - rad / r) ** 2
C:\Users\lpwu\AppData\Local\Temp\ipykernel_100360\1847673077.py:36: RuntimeWarning: invalid value encountered in multiply
  temp += frac * (1 - rad / r) ** 2


Exception: ('Problem Error: F can not be set, expected shape (32, 2) but provided (2, 0)', ValueError('cannot reshape array of size 0 into shape (32,2)'))